# Photon Number Estimates

### This notebook calculated the estimated signal of Next-like detectors 

In [1]:
import numpy as np

# Constants in all cases
E_qbb = 2400. # keV
E_kr = 41.5 # keV
dEdx_qbb_blob = 20 # keV/cm
dEdx_qbb_track = 7 # keV/cm
dEdx_kr = 20

In [2]:
def N_i(E, W_i=21.9):
    """
    Returns number of ionization electrons released 
    in a material with an ionization energy of W_i
    with an energy of E.
    
    Input: 
        E - energy in keV
        W_i - ionization energy in eV, default to GXe
    """
    return E*1000./W_i

In [3]:
print('N_i(Qbb) = '+str(int(N_i(E_qbb)))+', N_i(kr) = '+str(int(N_i(E_kr))))

N_i(Qbb) = 109589, N_i(kr) = 1894


In [4]:
def SAF_Plane(r_TP, d_EL, d_A):
    """
    Solid angle fraction of the circular plane a distance 
    d_EL+d_A from the source.
    
    Input: r_TP - radius of tracking plane [mm]
           d_EL - length of EL gap [mm]
           d_A - distance between anode and SiPMs [mm]
    """
    return 2*np.pi * (1/(4*np.pi)) * (1 - ((d_EL+d_A)/2.)/(np.sqrt(r_TP**2. + ((d_EL+d_A)/2.)**2.)))

In [5]:
def SAF_SiPM(s, d_EL, d_A, sipm_width=2.):
    """
    Solid angle fraction of square plane a distance
    d_EL+d_A from the source.
    Reference: https://vixra.org/pdf/2001.0603v1.pdf
    
    Input: s - size of SiPM [mm]
           d_EL - length of EL gap [mm]
           d_A - distance between anode and SiPMs [mm]
    """
    d = d_EL/2. + d_A - sipm_width
    return SAF_square(s, d)

def SAF_square(s, d):
    """
    Solid angle fraction of square plane a distance
    d from the source
    Reference: https://vixra.org/pdf/2001.0603v1.pdf
    
    Input: s - size of SiPM [mm]
           d - distance [mm]
    """
    num = s**2.
    denom = d**2. * np.sqrt(1. + 2.*s**2./d**2.)
    return (1./np.pi) * np.arctan( num / denom)

def SAF_square_approx(s, d):
    """
    Solid angle fraction of square plane, approx for small s 
    a distance d from the source.
    
    Input: s - size of SiPM [mm]
           d - distance from source [mm]
    """
    return s**2. / (4*np.pi*d**2.)

In [6]:
def Coverage(r_TP, s, N_sipms):
    """
    Coverage of tracking plane
    
    Input: r_TP - radius of tracking plane [mm]
           s - size of SiPM [mm]
           N_sipms - number of SiPMs [mm]
    """
    return s**2. * N_sipms / (np.pi * r_TP**2.)

In [7]:
def EL_yield(d_EL, Efield, P):
    """
    Returns the EL yeild, the number of photons per ionization
    electron crossing the EL gap.
    Reference: E.D.C.Freitas 2010, 'Secondary scintillation 
        yield in high-pressure xenon gas for neutrinoless double 
        beta decay  search'
        
    Input: d_EL - length of EL gap [mm], 
           Efield - electric field [kV/cm]
           P - pressure [bar]
    """
    red_efield = Efield/(P)
    red_ELyield = 151*red_efield-131
    ELyield = red_ELyield*(d_EL/10.)*P
    return ELyield

In [8]:
def N_d(E, eta, epsilon, f):
    """
    Returns the number of photons detected
    
    Inputs: E - energy of event [keV]
            eta - gain of EL gap
            epsilon - efficiencies, 
            f - fraction of light to planes (Solid angle fraction * Coverage)
    """
    return N_i(E) * eta * epsilon * f

In [9]:
def dN_dt(dE_dx, v_d, eta, epsilon, f_sipm):
    """
    Returns the number of photons detected
    
    Inputs: dE_dx - energy deposition [keV/mm]
            v_d - drift velocity [mm/us]
            eta - gain of EL gap
            epsilon - efficiencies, 
            f_sipm - fraction of light to center most SiPM
    """
    return N_i(dE_dx) * v_d * eta * epsilon * f_sipm

## NEXT Flex

In [10]:
# Constants in all flex simulations 
epsilon_flex = .286
r_TP_flex = 984./2. # mm
d_EL_flex = 10. # mm
d_A_flex = 15. # mm
sipm_width = 2.
v_drift_flex = 1. # mm/us
P_flex = 15 # bar
Efield_flex = 16. # kV/cm
eta_flex = EL_yield(d_EL_flex, Efield_flex, P_flex) # photons/e
C_pmts = 0.3 # need to check this
d_active_flex = 1160. # mm

In [11]:
# Simulation specific constants are included in the dictionary for that simulation
mcs = []
mcs.append({"size":1.3, "pitch":1.3, 'C':1.0, 'name':'1.3mm SiPM, 1.3mm pitch', "dir":"s1.3mmp1.3mm"})
mcs.append({"size":1.3, "pitch":7, 'C':0.012, 'name': '1.3mm SiPM, 7mm pitch',"dir": "s1.3mmp7mm"})
mcs.append({"size":1.3, "pitch":15, 'C':0.06, 'name': '1.3mm SiPM, 15mm pitch',"dir": "s1.3mmp15mm"})

mcs.append({"size":3, "pitch":3, 'C':1.0, 'name':'3mm SiPM, 3mm pitch', "dir":"s3mmp3mm"})
mcs.append({"size":3, "pitch":7, 'C':0.32, 'name': '3mm SiPM, 7mm pitch',"dir": "s3mmp7mm"})
mcs.append({"size":3, "pitch":15, 'C':0.07, 'name': '3mm SiPM, 15mm pitch', "dir": "s3mmp15mm"})

mcs.append({"size":6, "pitch":6, 'C':1.0, 'name':'6mm SiPM, 6mm pitch', "dir":"s6mmp6mm"})
mcs.append({"size":6, "pitch":15, 'C':0.28, 'name': '6mm SiPM, 15mm pitch', "dir": "s6mmp15mm"})

In [12]:
print('-------------------------')
print('NEXT Flex')
print('SiPM Solid angle fractions')
for mc in mcs:
    f_sipm_min = SAF_square(mc['size'], d_EL_flex + d_A_flex-sipm_width)
    f_sipm_max = SAF_square(mc['size'], d_A_flex-sipm_width)
    f_sipm_approx = SAF_square_approx(mc['size'], d_EL_flex + d_A_flex-sipm_width)
    print('-------------------------')
    print(mc['name']+': ')
    print('   f_SiPM range = '+str(f_sipm_min)+'-'+str(f_sipm_max))
    print('   f_SiPM(approx) = '+str(f_sipm_approx))
    print('   f_SiPM / f_SiPM(approx) = '+str(f_sipm_min / f_sipm_approx))
print('-------------------------')

-------------------------
NEXT Flex
SiPM Solid angle fractions
-------------------------
1.3mm SiPM, 1.3mm pitch: 
   f_SiPM range = 0.0010136701593998901-0.0031516345252008563
   f_SiPM(approx) = 0.0002542267049388499
   f_SiPM / f_SiPM(approx) = 3.9872686059621945
-------------------------
1.3mm SiPM, 7mm pitch: 
   f_SiPM range = 0.0010136701593998901-0.0031516345252008563
   f_SiPM(approx) = 0.0002542267049388499
   f_SiPM / f_SiPM(approx) = 3.9872686059621945
-------------------------
1.3mm SiPM, 15mm pitch: 
   f_SiPM range = 0.0010136701593998901-0.0031516345252008563
   f_SiPM(approx) = 0.0002542267049388499
   f_SiPM / f_SiPM(approx) = 3.9872686059621945
-------------------------
3mm SiPM, 3mm pitch: 
   f_SiPM range = 0.005325134961943662-0.016101185498196478
   f_SiPM(approx) = 0.0013538700263015673
   f_SiPM / f_SiPM(approx) = 3.9332689685808266
-------------------------
3mm SiPM, 7mm pitch: 
   f_SiPM range = 0.005325134961943662-0.016101185498196478
   f_SiPM(approx) = 0.

In [13]:
print('-------------------------')
print('NEXT Flex SiPMs')
for mc in mcs:
    f_flex = SAF_Plane(r_TP_flex, d_EL_flex, d_A_flex-sipm_width) * mc['C']
    f_sipm = SAF_SiPM(mc['size'], d_EL_flex, d_A_flex-sipm_width)
    print('-------------------------')
    print(mc['name']+': ')
    print('   N_d(Qbb) = '+str(int(N_d(E_qbb, eta_flex, epsilon_flex, f_flex))))
    print('   N_d(Kr) = '+str(int(N_d(E_kr, eta_flex, epsilon_flex, f_flex))))
    print('   dN_dt(Qbb, blob) = '+str(int(dN_dt(dEdx_qbb_blob, v_drift_flex, eta_flex, epsilon_flex, f_sipm))))
print('-------------------------')

-------------------------
NEXT Flex SiPMs
-------------------------
1.3mm SiPM, 1.3mm pitch: 
   N_d(Qbb) = 6902570
   N_d(Kr) = 119356
   dN_dt(Qbb, blob) = 245
-------------------------
1.3mm SiPM, 7mm pitch: 
   N_d(Qbb) = 82830
   N_d(Kr) = 1432
   dN_dt(Qbb, blob) = 245
-------------------------
1.3mm SiPM, 15mm pitch: 
   N_d(Qbb) = 414154
   N_d(Kr) = 7161
   dN_dt(Qbb, blob) = 245
-------------------------
3mm SiPM, 3mm pitch: 
   N_d(Qbb) = 6902570
   N_d(Kr) = 119356
   dN_dt(Qbb, blob) = 1273
-------------------------
3mm SiPM, 7mm pitch: 
   N_d(Qbb) = 2208822
   N_d(Kr) = 38194
   dN_dt(Qbb, blob) = 1273
-------------------------
3mm SiPM, 15mm pitch: 
   N_d(Qbb) = 483179
   N_d(Kr) = 8354
   dN_dt(Qbb, blob) = 1273
-------------------------
6mm SiPM, 6mm pitch: 
   N_d(Qbb) = 6902570
   N_d(Kr) = 119356
   dN_dt(Qbb, blob) = 4634
-------------------------
6mm SiPM, 15mm pitch: 
   N_d(Qbb) = 1932719
   N_d(Kr) = 33419
   dN_dt(Qbb, blob) = 4634
-------------------------


In [14]:
print('-------------------------')
print('NEXT Flex PMTs')
for mc in mcs:
    f_flex = SAF_Plane(r_TP_flex, d_EL_flex+d_active_flex, d_A_flex-sipm_width) * C_pmts
    print('-------------------------')
    print(mc['name']+': ')
    print('   N_d(Qbb) = '+str(int(N_d(E_qbb, eta_flex, 1.0, f_flex))))
    print('   N_d(Kr) = '+str(int(N_d(E_kr, eta_flex, 1.0, f_flex))))
print('-------------------------')

-------------------------
NEXT Flex PMTs
-------------------------
1.3mm SiPM, 1.3mm pitch: 
   N_d(Qbb) = 1713995
   N_d(Kr) = 29637
-------------------------
1.3mm SiPM, 7mm pitch: 
   N_d(Qbb) = 1713995
   N_d(Kr) = 29637
-------------------------
1.3mm SiPM, 15mm pitch: 
   N_d(Qbb) = 1713995
   N_d(Kr) = 29637
-------------------------
3mm SiPM, 3mm pitch: 
   N_d(Qbb) = 1713995
   N_d(Kr) = 29637
-------------------------
3mm SiPM, 7mm pitch: 
   N_d(Qbb) = 1713995
   N_d(Kr) = 29637
-------------------------
3mm SiPM, 15mm pitch: 
   N_d(Qbb) = 1713995
   N_d(Kr) = 29637
-------------------------
6mm SiPM, 6mm pitch: 
   N_d(Qbb) = 1713995
   N_d(Kr) = 29637
-------------------------
6mm SiPM, 15mm pitch: 
   N_d(Qbb) = 1713995
   N_d(Kr) = 29637
-------------------------


## DEMO

In [15]:
# Constants in demo
eta_demo = 1100
epsilon_demo = .22 * .88 # effective detection efficiency * anode transparaency 
r_TP_demo = 300./2. # mm
d_EL_demo = 5. # mm
d_A_demo = 2. # mm
s_demo = 1. # mm
N_sipms_demo = 256
C_demo = Coverage(r_TP_demo, s_demo, N_sipms_demo)
f_demo = SAF_Plane(r_TP_demo, d_EL_demo, d_A_demo) * C_demo
f_sipm_demo = SAF_SiPM(s_demo, d_EL_demo, d_A_demo)
v_drift_demo = 1.0

print('N_d(Qbb) = '+str(int(N_d(E_qbb, eta_demo, epsilon_demo, f_demo))))
print('N_d(kr) = '+str(int(N_d(E_kr, eta_demo, epsilon_demo, f_demo))))

N_d(Qbb) = 41275
N_d(kr) = 713


In [16]:
print('-------------------------')
print('DEMO SiPMs')
print('   N_d(Qbb) = '+str(int(N_d(E_qbb, eta_demo, epsilon_demo, f_demo))))
print('   N_d(Kr) = '+str(int(N_d(E_kr, eta_demo, epsilon_demo, f_demo))))
print('   dN_dt(Qbb, blob) = '+str(int(dN_dt(dEdx_qbb_blob, v_drift_demo, eta_demo, epsilon_demo, f_sipm))))
print('   dN_dt(Qbb, track) = '+str(int(dN_dt(dEdx_qbb_track, v_drift_demo, eta_demo, epsilon_demo, f_sipm))))
print('-------------------------')

-------------------------
DEMO SiPMs
   N_d(Qbb) = 41275
   N_d(Kr) = 713
   dN_dt(Qbb, blob) = 7651
   dN_dt(Qbb, track) = 2678
-------------------------


In [17]:
print('-------------------------')
print('DEMO PMTs')
print('   N_d(Qbb) = '+str(int(N_d(E_qbb, eta_demo, 1.0, f_demo))))
print('   N_d(Kr) = '+str(int(N_d(E_kr, eta_demo, 1.0, f_demo))))
print('-------------------------')

-------------------------
DEMO PMTs
   N_d(Qbb) = 213199
   N_d(Kr) = 3686
-------------------------


## NEW

SiPMs see about 22,000 pes total for a kr event. This was done roughly with run 7470, not cuts.

In [18]:
# Constants in demo
P_new = 10. # bar 
Efield_new = 17. # kV/cm
epsilon_new = .07 # should edit this somehow
r_TP_new = 454./2. # mm
d_EL_new = 10. # mm
d_A_new = 2. # mm # to fix
eta_new = EL_yield(d_EL_new, Efield_new, P_new)
s_new = 1.3 # mm
N_sipms_new = 1792
C_new = Coverage(r_TP_new, s_new, N_sipms_new)
f_new = SAF_Plane(r_TP_new, d_EL_new, d_A_new) * C_new
f_sipm_new = SAF_SiPM(s_new, d_EL_new, d_A_new)
v_drift_new = 1. # mm/us
d_drift_new = 664.5 # mm

N_d_data = 22000 # roughly average total signal in SiPMs from kr events in NEW in run 7470
eff_new = N_d_data / (N_i(E_kr, W_i=21.9) * eta_new * f_new)

In [19]:
print('-------------------------')
print('NEW SiPMs')
print('   N_d(Qbb) = '+str(int(N_d(E_qbb, eta_new, epsilon_new, f_new))))
print('   N_d(Kr) = '+str(int(N_d(E_kr, eta_new, epsilon_new, f_new))))
print('   dN_dt(Qbb, blob) = '+str(int(dN_dt(dEdx_qbb_blob, v_drift_new, eta_new, epsilon_new, f_sipm))))
print('-------------------------')

-------------------------
NEW SiPMs
   N_d(Qbb) = 87814
   N_d(Kr) = 1518
   dN_dt(Qbb, blob) = 3161
-------------------------


In [20]:
print('-------------------------')
print('NEW PMTs')
print('   N_d(Qbb) = '+str(int(N_d(E_qbb, eta_new, 1.0, f_new))))
print('   N_d(Kr) = '+str(int(N_d(E_kr, eta_new, 1.0, f_new))))
print('-------------------------')

-------------------------
NEW PMTs
   N_d(Qbb) = 1254486
   N_d(Kr) = 21692
-------------------------
